Primero, debe ser claro que para identificar el home location solo necesitas un stop table y una columna que tenga un location id, o un geohash o cualquier teselación. Por qué no hacemos que el punto de partida sea un stop table? Cargar un stop table que tenga trayectorias de todo un mes.
7:38
o más, digamos, para 5 usuarios.
7:39
Entonces el notebook puede ser sobre el módulo visit_attribution en general (más o menos lo que estás haciendo ahí de todas formas).
7:43
Lo primero es mostrar como se puede usar point_in_polygon para atribuir visitas a ubicaciones. Segundo, que hay otro método que usa la moda de los pings en el stop (para el cuál habría que cargar los labels también), y visualizar eso.
Parte dos es como se puede usar ese output muy fácilmente para estimar la casa:
Calcular el duration at night de cada stop. Equivalente a la columna "duration" pero sólo contando minutos que ocurran entre los dos parámetros de tiempo. Explicas eso.
Luego se filtra, y se cuentan días y semanas. Y ahí podemos mostrar el output que tienes en la última celda.
Finalmente, una función muy sencilla, dados unos parámetros como dusk_time, dawn_time, min_nights, min_weeks, min_night_dwell, window_weeks te retorna la ubicación más frecuente en la noche que cumpla esas restricciones.
(edited)
7:44
Como en el otro notebook, lo más importante es que haya un parrafito donde explicamos un poco qué está pasando.
7:44
Así como hiciste en el de stop detection.
7:45
Y habrá algunas visualizaciones o algo, si.

# Tutorial X: Home Attribution from Stop Table

This notebook shows how to process a stop table for detecting homes, using ```nomad```. 



Stop detection is an important step in pre-processing trajectory data and in making sense of trajectories by grouping together pings that reflect stationary behavior. The output of stop-detection algorithms is commonly a "stop table", indicating when a stop started, its duration, and a pair of coordinates that approximates the location of the group of pings (typically the centroid). Alternatively, ```nomad``` allows users to retrieve a cluster label for each ping (useful for plotting, for example).

In [161]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import geopandas as gpd

In [141]:
import nomad.io.base as loader
import nomad.visit_attribution as va
import nomad.stop_detection.lachesis as LACHESIS

In [162]:
traj_cols = {'uid':'uid',
             'x':'x',
             'y':'y',
             'timestamp':'timestamp'}

diaries_df = loader.from_file("../nomad/data/diaries", format="parquet", traj_cols=traj_cols,
                       parse_dates=True)
sparse_df = loader.from_file("../nomad/data/sparse_traj/", format="parquet", traj_cols=traj_cols,
                      parse_dates=True)
poi_table = gpd.read_file('garden_city.geojson')

# Reproject from gc_coords to web mercator
sparse_df.loc[:,'x'] = (sparse_df['x'] - 4265699)/15
sparse_df.loc[:,'y'] = (sparse_df['y'] + 4392976)/15

diaries_df.loc[:,'x'] = (diaries_df['x'] - 4265699)/15
diaries_df.loc[:,'y'] = (diaries_df['y'] + 4392976)/15

# Select data from 1 user
user = diaries_df.uid.unique()[0]
user_sample = sparse_df.loc[sparse_df['uid'] == user]

user_sample

,uid,timestamp,longitude,latitude,x,y,local_timestamp,ha,date
11327,admiring_curie,1704148860,38.321317,-36.666853,14.031005,13.632870,2024-01-02 00:41:00+02:00,11.25,2024-01-02
11328,admiring_curie,1704149640,38.321373,-36.666809,14.449639,14.038498,2024-01-02 00:54:00+02:00,11.25,2024-01-02
11329,admiring_curie,1704150900,38.321450,-36.666825,15.022904,13.891453,2024-01-02 01:15:00+02:00,11.25,2024-01-02
11330,admiring_curie,1704151080,38.321470,-36.666668,15.171014,15.337402,2024-01-02 01:18:00+02:00,11.25,2024-01-02
11331,admiring_curie,1704151380,38.321387,-36.666771,14.551866,14.383345,2024-01-02 01:23:00+02:00,11.25,2024-01-02
...,...,...,...,...,...,...,...,...,...
143046,admiring_curie,1705740660,38.319985,-36.666497,4.144425,16.921279,2024-01-20 10:51:00+02:00,11.25,2024-01-20
143047,admiring_curie,1705741440,38.320014,-36.666660,4.366099,15.416760,2024-01-20 11:04:00+02:00,11.25,2024-01-20
143048,admiring_curie,1705744320,38.320025,-36.666459,4.446864,17.270041,2024-01-20 11:52:00+02:00,11.25,2024-01-20
143049,admiring_curie,1705744500,38.320077,-36.666465,4.828045,17.222850,2024-01-20 11:55:00+02:00,11.25,2024-01-20


In [165]:
poi_table

,building_id,type,geometry
0,p-x13-y11,park,"POLYGON ((13 9, 13 13, 9 13, 9 9, 13 9))"
1,h-x8-y8,home,"POLYGON ((8 7, 8 9, 7 9, 7 7, 8 7))"
2,h-x9-y8,home,"POLYGON ((10 7, 10 8, 8 8, 8 7, 10 7))"
3,h-x10-y8,home,"POLYGON ((11 7, 11 8, 10 8, 10 7, 11 7))"
4,h-x11-y8,home,"POLYGON ((12 7, 12 8, 11 8, 11 7, 12 7))"
...,...,...,...
101,r-x3-y7,retail,"POLYGON ((3 7, 3 8, 1 8, 1 7, 3 7))"
102,r-x0-y5,retail,"POLYGON ((2 4, 2 7, 1 7, 1 4, 2 4))"
103,r-x3-y6,retail,"POLYGON ((3 6, 3 7, 2 7, 2 6, 3 6))"
104,r-x3-y5,retail,"POLYGON ((3 5, 3 6, 2 6, 2 5, 3 5))"


In [163]:
DUR_MIN=5
DT_MAX=60
DELTA_ROAM=100

traj_cols = {'uid':'uid',
             'x':'x',
             'y':'y',
             'datetime':'local_timestamp'}

stop_table_lachesis = LACHESIS.lachesis(traj=user_sample,
                                        dur_min=DUR_MIN,
                                        dt_max=DT_MAX,
                                        delta_roam=DELTA_ROAM,
                                        traj_cols=traj_cols,
                                        keep_col_names=False,
                                        complete_output=True,
                                        datetime = 'local_timestamp')

labels_lachesis = LACHESIS._lachesis_labels(traj=user_sample,
                                            dur_min=DUR_MIN,
                                            dt_max=DT_MAX,
                                            delta_roam=DELTA_ROAM,
                                            traj_cols=traj_cols,
                                            datetime = 'local_timestamp')
labels_lachesis.name = 'cluster'

pred_lachesis = va.point_in_polygon(traj=user_sample,
                 labels=labels_lachesis,
                 stop_table=stop_table_lachesis,
                 poi_table=poi_table,
                 traj_cols=traj_cols,
                 is_datetime=True,
                 is_long_lat=False)

pred_lachesis

/Users/andresmondragon/nomad/nomad/io/base.py:55: UserWarning: Trajectory column 'uid' specified for 'uid' not found in DataFrame.
  warnings.warn(f"Trajectory column '{value}' specified for '{key}' not found in DataFrame.")
/Users/andresmondragon/nomad/nomad/io/base.py:55: UserWarning: Trajectory column 'local_timestamp' specified for 'datetime' not found in DataFrame.
  warnings.warn(f"Trajectory column '{value}' specified for '{key}' not found in DataFrame.")


,start_datetime,end_datetime,duration,x,y,diameter,n_pings,max_gap,location
0,2024-01-02 00:41:00+02:00,2024-01-02 01:26:00+02:00,45,14.551866,14.383345,2.050621,6,21,h-x13-y13
1,2024-01-02 03:17:00+02:00,2024-01-02 06:36:00+02:00,199,14.359300,14.630957,2.713152,20,34,h-x13-y13
2,2024-01-02 19:30:00+02:00,2024-01-02 21:07:00+02:00,97,14.193728,9.586870,2.804123,3,57,p-x13-y11
3,2024-01-03 05:52:00+02:00,2024-01-03 09:39:00+02:00,227,4.794544,10.901612,11.248490,14,47,h-x8-y10
4,2024-01-03 15:10:00+02:00,2024-01-03 15:44:00+02:00,34,4.114724,16.135475,1.451006,3,21,w-x3-y16
5,2024-01-04 20:39:00+02:00,2024-01-05 03:20:00+02:00,401,14.559428,8.326059,2.000332,35,42,h-x13-y8
6,2024-01-05 15:33:00+02:00,2024-01-05 15:58:00+02:00,25,4.753138,6.431516,1.014941,4,17,w-x3-y6
7,2024-01-06 04:19:00+02:00,2024-01-06 04:40:00+02:00,21,11.979128,6.937445,1.104563,3,19,h-x11-y8
8,2024-01-06 13:33:00+02:00,2024-01-06 14:36:00+02:00,63,4.655410,16.576277,6.712630,6,45,w-x3-y13
9,2024-01-06 21:46:00+02:00,2024-01-07 00:02:00+02:00,136,14.359073,8.486195,1.859656,13,24,h-x13-y8


In [164]:
va.night_stops(pred_lachesis, user)

,id,location,night_count,week_count
0,admiring_curie,h-x10-y8,1,1
1,admiring_curie,h-x11-y8,1,1
2,admiring_curie,h-x13-y12,1,1
3,admiring_curie,h-x13-y13,1,1
4,admiring_curie,h-x13-y8,5,2
5,admiring_curie,h-x15-y13,1,1
6,admiring_curie,h-x8-y8,1,1
7,admiring_curie,p-x13-y11,3,2
